In [ ]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

batch_size = 128
epochs = 10
seed = 1
log_interval = 10
torch.manual_seed(seed)
device = "cpu"

In [ ]:
from Tars.distributions import CategoricalModel
from Tars.models import ML

In [ ]:
kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

In [ ]:
x_dim = 784
y_dim = 10

# discriminative model p(y|x)
class Discriminative(CategoricalModel):
    def __init__(self):
        super(Discriminative, self).__init__(cond_var=["x"], var=["y"])
        self.fc1 = nn.Linear(x_dim, 512)
        self.fc2 = nn.Linear(512, 512)        
        self.fc3 = nn.Linear(512, y_dim)

    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        h3 = F.softmax(self.fc3(h2))
        
        return h3

In [ ]:
p = Discriminative()
p.to(device)

In [ ]:
model = ML(p, optim.Adam, {"lr":1e-3})

In [ ]:
def train(epoch):
    train_loss = 0
    for batch_idx, (data_x, data_y) in enumerate(train_loader):
        data_x = data_x.view(-1, 784).to(device)
        data_y = torch.eye(10)[data_y].to(device)
        log_like, loss = model.train({"x": data_x,"y": data_y})
        train_loss += loss
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data_x), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item()))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [ ]:
def test(epoch):
    test_loss = 0
    for i, (data_x, data_y) in enumerate(test_loader):
        data_x = data_x.view(-1, 784).to(device)
        data_y = torch.eye(10)[data_y].to(device)
        log_like, loss = model.test({"x": data_x,"y": data_y})
        test_loss += loss

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [10]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)

Train Epoch: 3 [30720/60000 (51%)]	Loss: 0.049872
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.073125
Train Epoch: 3 [33280/60000 (55%)]	Loss: 0.054717
Train Epoch: 3 [34560/60000 (58%)]	Loss: 0.047753
Train Epoch: 3 [35840/60000 (60%)]	Loss: 0.036453
Train Epoch: 3 [37120/60000 (62%)]	Loss: 0.020681
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.023433
Train Epoch: 3 [39680/60000 (66%)]	Loss: 0.066916
Train Epoch: 3 [40960/60000 (68%)]	Loss: 0.095758
Train Epoch: 3 [42240/60000 (70%)]	Loss: 0.036439
Train Epoch: 3 [43520/60000 (72%)]	Loss: 0.069195
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.050549
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.064636
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.103772
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.026690
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.035649
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.093868
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.082531
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.089288
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.043185
